In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time

def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser("chrome", **executable_path, headless=False)

In [ ]:
def scrape():
    browser = init_browser()
    # create mars_data dict that we can insert into mongo
    mars_dic = {}
    
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, 'html.parser')

    article = soup.find("div", class_="list_text")
    news_title = article.find("div", class_="content_title").text
    news_p = article.find("div", class_="article_teaser_body").text
    
    # add the text to mars_dic
    mars_dic["news_title"] = news_title
    mars_dic["news_p"] = news_p
    
    
    # Visit the JPL Mars Space Images URL
    url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url2)

    html_2 = browser.html
    soup_2 = bs(html_2, 'html.parser')

    image_url_find = soup_2.find("article", class_="carousel_item")["style"]
    image_url_parts = image_url_find.split("'")[1]
    
    featured_image_url = "https://www.jpl.nasa.gov" + image_url_parts
    mars_dic["features"] = featured_image_url
    
      #  twitter URL
    # setup the chromedriver path
    # Visit the Mars weather twitter URL
    url3 = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url3)

    # Scrape the browser into soup and use soup to find the latest Mars weather
    html_3 = browser.html
    soup_3 = bs(html_3, 'html.parser')

    mars_weather = soup_3.find('div', class_='js-tweet-text-container').find('p').text
    mars_dic["Weather"] = mars_weather
    
    #Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc. 
    #Use Pandas to convert the data to a HTML table string.
    url4 = 'https://space-facts.com/mars/'

    mars_tables = pd.read_html(url4)
    df = mars_tables[0]
    df.columns = ['Parameter', 'Value']
    df.set_index('Parameter', inplace=True)


    # Convert the data to a HTML table string.
    html_table = df.to_html().replace('\n', '')

    mars_dic["HTML"]=html_table
    
    
    #Hemisphere
    #Prepare empty list to store dictionary of URL and titles
    hemisphere_image_urls = []

    #Loop 
    for i in range(4):

        browser.find_by_css("a.product-item h3")[i].click()
        html = browser.html
        soup = bs(html, 'lxml')

        #Store in a variable
        image = soup.find_all('img', class_='wide-image')[0]['src']
        url = 'https://astrogeology.usgs.gov' + image

        #Store the title into a varialbe and removed enhanced
        title = soup.find('h2', class_='title').text.replace(' Enhanced', '')

        #Append URL and title to a dictionary and append to list
        hemisphere_image_urls.append({'title': title,
                                 'img_url': url})

        #Back to previous page to loop through other hemispheres.
        browser.click_link_by_text('Back')
        
        mars_dic["hemisphere"] = hemisphere_image_urls
   